In [2]:
from os import listdir, stat
from PIL import Image, ImageFile
import numpy as np
import tensorflow as tf
import math
from sklearn.model_selection import train_test_split
tf.test.is_gpu_available(
    cuda_only=True,
    min_cuda_compute_capability=None
)

True

In [3]:
# Load pictures
path = "D:/Unity/Projects/screenshots"
directory = listdir(path)
X = np.empty(shape=(len(directory),108,192,3))
y = y=np.zeros(shape=(len(directory)))
for i,image in enumerate(directory):
    imagepath = path + "/" + image
    img = Image.open(imagepath)
    X[i]=np.asarray(img)
    # Read labels
    binary="000"
    if image[-5]=="1":
        binary="1"+binary[1:]
    if image[-6]=="1":
        binary=binary[0]+"1"+binary[2]
    if image[-7]=="1":
        binary=binary[:2]+"1"
    # Convert from multilabel to multiclass classification:
    y[i]=int(binary, 2)

In [4]:
y = np.int_(y)
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=6)

In [5]:
def cnn_model_fn(features, labels, mode):

    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 108, 192, 3])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=10,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    #output shape [-1,108,192,32]

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[3,3], strides=[5,9])
    #output dim=((D-Fd+2*P )/Sd)+1
    #output shape [-1,22,22,10]

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=20,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    #output shape [-1,22,22,20]
    
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    #output dim=((D-Fd+2*P )/Sd)+1
    #output shape [-1,11,11,20]

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 11 * 11 * 20])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, 
                                rate=0.4, 
                                training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=8)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [10]:
def cnn_model_fn2(features, labels, mode):

    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 108, 192, 3])

    # Custom Convolution for Vertical Edge Detection (Sobel Filter)
    w = np.array([[-1,0,1],[-2,0,2],[-1,0,1]],dtype="float32")
    kernel = np.array([[w,w,w],[w,w,w],[w,w,w]])
    edge_v = tf.nn.conv2d(
        input=input_layer,
        filter=kernel,
        strides=[1,1,1,1],
        padding="SAME")
    #output shape [-1,108,192,3]
    
    # Custom Convolution for Horizontal Edge Detection (Sobel Filter)
    w = np.array([[1,2,1],[0,0,0],[-1,-2,-1]],dtype="float32")
    kernel = np.array([[w,w,w],[w,w,w],[w,w,w]])
    edge_h = tf.nn.conv2d(
        input=input_layer,
        filter=kernel,
        strides=[1,1,1,1],
        padding="SAME")
    #output shape [-1,108,192,3]
        
    sum1 = edge_v+edge_h+input_layer
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=sum1,
      filters=10,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    #output shape [-1,108,192,32]

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[3,3], strides=[5,9])
    #output dim=((D-Fd+2*P )/Sd)+1
    #output shape [-1,22,22,10]

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=20,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    #output shape [-1,22,22,20]
    
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    #output dim=((D-Fd+2*P )/Sd)+1
    #output shape [-1,11,11,20]

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 11 * 11 * 20])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, 
                                rate=0.4, 
                                training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=8)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [18]:
# Set up Estimator
computer_vision = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./model4")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023A3781B240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
computer_vision.train(
    input_fn=train_input_fn,
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./model4\model.ckpt.
INFO:tensorflow:loss = 54.64387512207031, step = 0
INFO:tensorflow:global_step/sec: 2.57893
INFO:tensorflow:loss = 2.060713291168213, step = 100 (38.770 sec)
INFO:tensorflow:global_step/sec: 2.59581
INFO:tensorflow:loss = 2.027195692062378, step = 200 (38.524 sec)
INFO:tensorflow:global_step/sec: 2.23661
INFO:tensorflow:loss = 1.9722405672073364, step = 300 (44.726 sec)
INFO:tensorflow:global_step/sec: 1.81861
INFO:tensorflow:loss = 1.9149305820465088, step = 400 (54.971 sec)
INFO:tensorflow:global_step/sec: 1.52582
INFO:tensorflow:loss = 1.7438738346099854, step = 500 (65.551 sec)
INFO:tensorflow:global_step/sec: 1.57122
INFO:tensorflow:loss = 1.618357539176941, step = 600 (63.638 sec)


In [20]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = computer_vision.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-23-16:01:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model4\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-23-16:01:10
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.5469954, global_step = 1000, loss = 1.224402
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: ./model4\model.ckpt-1000
{'accuracy': 0.5469954, 'loss': 1.224402, 'global_step': 1000}
